In [1]:
import main
import utils

In [2]:
client = main.instantiate_db()

Skipping class creation


In [3]:
client.schema.delete_all()

In [4]:
from pathlib import Path
filepath = Path("./data/kubernetes_concepts_overview.txt")
src_data = main.SourceData(
    source_path=filepath,
    source_text=utils.load_data(filepath)
)

In [5]:
main.add_to_weaviate(src_data, client)

13

In [6]:
for wiki_title in [
    "Database",
    "Vector database",
    "Containerization (computing)",
    "Graphite",
]:
    src_data = main.SourceData(
        source_path=wiki_title,
        source_text=utils.load_wiki_page(wiki_title)
    )
    main.add_to_weaviate(src_data, client)

In [7]:
collection = utils.Collection(client, main.WV_CLASS)

In [8]:
collection.text_search("vector database")

[{'_additional': {'distance': 0.16120988},
  'body': 'A Vector Database is a type of specialized database designed to handle vector embeddings. These embeddings are a form of data representation that conveys crucial semantic information for a variety of applications, including large language models, generative AI, and semantic search. AI models generate these embeddings, each encompassing numerous attributes or features. These features represent different dimensions of the data and are essential for understanding patterns, relationships, and underlying structures.',
  'source_path': 'Vector database'},
 {'_additional': {'distance': 0.23336434},
  'body': 'data as rows and columns in a series of tables, and the vast majority use SQL for writing and querying data. In the 2000s, non-relational databases became popular, collectively referred to as NoSQL, because they use different query languages.',
  'source_path': 'Database'},
 {'_additional': {'distance': 0.24594474},
  'body': 'In comp

In [9]:
collection.generate_summary("database")

'A database is an organized collection of data stored and accessed electronically. It can be stored on a file system or hosted on computer clusters or cloud storage. A database management system (DBMS) is the software that interacts with end users, applications, and the database itself to capture and analyze the data. Relational databases became dominant in the 1980s, while non-relational databases (NoSQL) became popular in the 2000s. A Vector Database is a specialized database designed to handle vector embeddings, which are used for various applications such as large language models and generative AI.'

In [10]:
collection.suggest_topics_with_weaviate("nosql database")

'- NoSQL databases\n- Query languages for NoSQL databases\n- Differences between NoSQL and relational databases\n- Benefits of using NoSQL databases\n- Examples of popular NoSQL databases\n- Use cases for NoSQL databases'